Resize Images to desired size

In [1]:
image_size = 224

In [2]:
# Mount Drive 
from google.colab import drive

drive.mount('/content/gdrive')


#Change to the correct directory
import os
os.chdir('/content/gdrive/My Drive/junior/442')

!ls
!pwd


#Make imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, models, transforms
from PIL import Image
import numpy as np
import os
import numpy as np
import PIL
from PIL import Image
import cv2 as cv
from cv2 import imread
import time
import copy


# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")


Mounted at /content/gdrive
'Colab Notebooks'   png   png224   sketches_png.zip
/content/gdrive/My Drive/junior/442
Using the GPU!


In [ ]:
#!wget http://cybertron.cg.tu-berlin.de/eitz/projects/classifysketch/sketches_png.zip
!rm -r png
!unzip sketches_png.zip && rm sketches_png.zip

In [12]:
# The following 3 funtions are adapted from https://github.com/wayneb0t/sketch-recognition

def load_image(path):
  """
  Load images from the given path
  """
  im_data = imread(path, cv.IMREAD_GRAYSCALE)
  return im_data

def resize_images(res=128):
  """
  Resize all images into a res x res img (default is 128x128)
  """
  root_dir = "png/"
  new_dir = "png{}/".format(res)
  try:
      os.mkdir(new_dir, 755)
  except:
      pass
  #for each class type
  for classType in sorted(os.listdir(root_dir)):
      #if already created, skip
      if os.path.isfile(root_dir + classType):
          continue
     
      label_path = root_dir + classType + "/"
      new_path = new_dir + classType + "/"
      
      try:
          os.mkdir(new_path, 755)
      except:
          pass
      #For each file in class
      for im_file in os.listdir(label_path):
          im_path = label_path + im_file
          new_im_path = new_path + im_file
          
          img = Image.open(im_path)
          img = img.resize((res, res), PIL.Image.BILINEAR)
          img.save(new_im_path)
          img.close()

def move_data(num_classes=250, res=128, saveFile=False):
    """
    Generates the datasets with 128 (or 64) training examples, 8 validation examples,
    and 8 testing examples per class.
    
    Args:
        num_classes: the number of classes to load (for smaller datasets)
        res: the resolution of the output arrays (N x res x res)
        flip: whether or not to generate additional training examples by horizontally
              flipping the provided images
        color_invert: whether or not to invert B&W values
    """
    root_dir = "png{}/".format("" if res is None else res)
    output_dir = "png{}output/".format("" if res is None else res)
    num_train = 48
    num_val = 16
    num_test = 16
    
    labels = []
    
    X_train = np.zeros((num_classes * num_train, res, res, 1), dtype=np.float32)
    y_train = np.repeat(np.arange(num_classes), num_train)
    
    X_val = np.zeros((num_classes * num_val, res, res, 1), dtype=np.float32)
    y_val = np.repeat(np.arange(num_classes), num_val)
    
    X_test = np.zeros((num_classes * num_test, res, res, 1), dtype=np.float32)
    y_test = np.repeat(np.arange(num_classes), num_test)
    
    classes = 0
    train_index = 0
    val_index = 0
    test_index = 0
    
    for node in sorted(os.listdir(root_dir)):
        if os.path.isfile(root_dir + node):
            continue
        
        labels.append(node)
        label_path = root_dir + node + "/"
        
        num_images = 0
        for im_file in sorted(os.listdir(label_path)):
            im_data = load_image(label_path + im_file).reshape(res, res, 1)
            
            if num_images < num_train:
                typeStr = "train"
                X_train[train_index] = im_data
                train_index += 1
                    
            elif num_images < num_train + num_val:
                X_val[val_index] = im_data
                typeStr = "val"
                val_index += 1
            else:
                X_test[test_index] = im_data
                typeStr = "test"
                test_index += 1

            if (saveFile):
              im_path = label_path + im_file
              new_im_path = output_dir + typeStr + "/" + node + '/' + im_file
              try:
                  os.mkdir(output_dir, 755)
              except:
                  pass
              try:
                  os.mkdir(output_dir + typeStr + "/", 755)
              except:
                  pass
              try:
                  os.mkdir(output_dir + typeStr + "/" + node + '/', 755)
              except:
                  pass
              img = Image.open(im_path)
              img = img.resize((res, res), PIL.Image.BILINEAR)
              img.save(new_im_path)
              img.close()



            num_images += 1
                
        classes += 1
        if classes == num_classes:
            break

    return X_train, y_train, X_val, y_val, X_test, y_test, labels


In [13]:
#!rm -r png128
##actually resize the images
resize_images(res=image_size)


In [14]:
# Organize data for dataloader function
# Moved into
#  Model
#   \ train
#     \ classes
#     \  ...
#   \ val
#     \ classes
#     \  ...
#   \ test
#     \ classes
#     \  ...

#Will return them as dataloaders which are not used
X_train, y_train, X_val, y_val, X_test, y_test, labels = move_data(num_classes=250, res=224,saveFile=True)
# X_train = 48*250 (12000) x 128 x 128 x 1
# X_val = 16*250 (4000) x 128 x 128 x 1
# X_test = 16*250 (4000) x 128 x 128 x 1

print(len(y_train)) # 12000
print(len(y_val)) # 4000
print(len(y_test)) # 4000
print(len(labels)) # 250

12000
4000
4000
250


In [20]:
# Double check done correctly


#should all be 250
!ls -l ./png | grep "^d" | wc -l
!ls -l ./png224 | grep "^d" | wc -l
!ls -l ./png224output/train | grep "^d" | wc -l
!ls -l ./png224output/test | grep "^d" | wc -l
!ls -l ./png224output/val | grep "^d" | wc -l

# 48 16 16
!ls -l ./png224output/train/airplane | egrep -c '^-'
!ls -l ./png224output/test/airplane | egrep -c '^-'
!ls -l ./png224output/val/airplane | egrep -c '^-'

# all 80
!ls -l ./png224/airplane | egrep -c '^-'
!ls -l ./png224/zebra | egrep -c '^-'
!ls -l ./png/zebra | egrep -c '^-'
!ls -l ./png/airplane | egrep -c '^-'



250
250
250
250
250
48
16
16
80
80
80
80
